In [11]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
import pandas as pd

In [12]:
train = pd.read_csv("../model_data/train_day.csv")

C:\Users\Ambarish Ganguly\AppData\Local\Temp\ipykernel_17008\3024732326.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../model_data/train_day.csv")


In [13]:
train.columns

Index(['day', 'energy_sum', 'temperatureMin', 'temperatureMax', 'uvIndex',
       'dewPoint', 'Type', 'month', 'Type_Binary'],
      dtype='object')

In [14]:
df = pd.DataFrame()

df["ds"] = train["day"]
df["y"] = train["energy_sum"]
df["unique_id"] = "Energy"

In [15]:
df.head()

,ds,y,unique_id
0,2013-01-03,10.074,Energy
1,2013-01-06,10.293,Energy
2,2013-01-07,9.439,Energy
3,2013-01-11,10.979,Energy
4,2013-01-12,10.586,Energy


In [16]:
sf = StatsForecast(
    models=[AutoARIMA()],
    freq='D'
)
sf.fit(df)

StatsForecast(models=[AutoARIMA])

In [17]:
forecast_df = sf.predict(h=30, level=[90])
forecast_df.tail()

,unique_id,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
25,Energy,2014-01-26,11.946,-91.906654,115.798654
26,Energy,2014-01-27,11.946,-93.884977,117.776977
27,Energy,2014-01-28,11.946,-95.826992,119.718992
28,Energy,2014-01-29,11.946,-97.734627,121.626627
29,Energy,2014-01-30,11.946,-99.609645,123.501645


In [18]:
forecast_df

,unique_id,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,Energy,2014-01-01,11.946,-8.421181,32.313181
1,Energy,2014-01-02,11.946,-16.857544,40.749544
2,Energy,2014-01-03,11.946,-23.330992,47.222992
3,Energy,2014-01-04,11.946,-28.788362,52.680362
4,Energy,2014-01-05,11.946,-33.596401,57.488401
5,Energy,2014-01-06,11.946,-37.943201,61.835201
6,Energy,2014-01-07,11.946,-41.940496,65.832496
7,Energy,2014-01-08,11.946,-45.661087,69.553087
8,Energy,2014-01-09,11.946,-49.155543,73.047543
9,Energy,2014-01-10,11.946,-52.460682,76.352682
